In [34]:
import pandas as pd
pd.set_option('display.max_rows', 1000)

In [35]:
# read in october 2019 mta data
df1 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_191026.txt')
df2 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_191019.txt')
df3 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_191012.txt')
df4 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_191005.txt')

In [36]:
dataframes = [df1, df2, df3, df4]

In [37]:
def combine_dfs_add_time(dataframe_list):
    """
    feed in a list of turnstyle dataframes
    returns the combined datafrae with columns for date time and day of week
    """
    # concatenate the dataframes into one
    df = pd.concat(dataframe_list, ignore_index=True)
    
    # rename the exits field
    df = df.rename(columns={'EXITS                                                               ': 'EXITS'})
    
    # create a new column that combines the day and time into one and makes it a datetime object
    df["DATE_TIME"] =  pd.to_datetime(df["DATE"] +" "+ df["TIME"])
    
    # add in a day of the week column
    df["DAY_INT"] = df["DATE_TIME"].dt.dayofweek
    
    # create a mapper to map the day of the week nubers to actual string values
    day_dict = {
        0: 'Monday',
        1: 'Tuesday',
        2: 'Wednesday',
        3: 'Thursday',
        4: 'Friday',
        5: 'Saturday',
        6: 'Sunday'
    }
    
    # add that day of the week string column
    df["DAY_STR"] = df["DAY_INT"].map(day_dict)
    
    return df

In [39]:
combined_df = combine_dfs_add_time(dataframes)
combined_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY_INT,DAY_STR
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/19/2019,00:00:00,REGULAR,7238905,2452500,2019-10-19 00:00:00,5,Saturday
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/19/2019,04:00:00,REGULAR,7238924,2452505,2019-10-19 04:00:00,5,Saturday
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/19/2019,08:00:00,REGULAR,7238945,2452536,2019-10-19 08:00:00,5,Saturday
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/19/2019,12:00:00,REGULAR,7239029,2452602,2019-10-19 12:00:00,5,Saturday
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/19/2019,16:00:00,REGULAR,7239280,2452651,2019-10-19 16:00:00,5,Saturday


In [40]:
def add_entry_and_exit_differences(df):
    """
    Takes in the dataframe with the date time column
    returns a data frame with a entry and exit diff column
    these new columns tell us how many people exuted/entered in that time period
    """
    
    # sort the dataframe by turnstyle date
    ordered_date_df = df.sort_values(by=['STATION', 'SCP','UNIT','C/A', 'DATE_TIME'])
    
    """
    group by station, scp, unit, and c/a to get the individual counters 
    then take the difference in entries to get entry changes on each timestamp
    """
    ordered_date_df['ENTRIES_DIFF']=ordered_date_df.groupby(['STATION', 'SCP','UNIT','C/A'])['ENTRIES'].diff().fillna(0)
    
    """
    group by station, scp, unit, and c/a to get the individual counters 
    then take the difference in exits to get exit changes on each timestamp
    """
    ordered_date_df['EXIT_DIFF']=ordered_date_df.groupby(['STATION', 'SCP', 'UNIT', 'C/A'])['EXITS'].diff().fillna(0)
    
    return ordered_date_df

In [41]:
combined_df = add_entry_and_exit_differences(combined_df)
combined_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY_INT,DAY_STR,ENTRIES_DIFF,EXIT_DIFF
651330,H007,R248,00-00-00,1 AV,L,BMT,09/28/2019,00:00:00,REGULAR,15011834,16775459,2019-09-28 00:00:00,5,Saturday,0.0,0.0
651331,H007,R248,00-00-00,1 AV,L,BMT,09/28/2019,04:00:00,REGULAR,15011834,16775459,2019-09-28 04:00:00,5,Saturday,0.0,0.0
651332,H007,R248,00-00-00,1 AV,L,BMT,09/28/2019,08:00:00,REGULAR,15011834,16775461,2019-09-28 08:00:00,5,Saturday,0.0,2.0
651333,H007,R248,00-00-00,1 AV,L,BMT,09/28/2019,12:00:00,REGULAR,15011834,16775480,2019-09-28 12:00:00,5,Saturday,0.0,19.0
651334,H007,R248,00-00-00,1 AV,L,BMT,09/28/2019,16:00:00,REGULAR,15011834,16775500,2019-09-28 16:00:00,5,Saturday,0.0,20.0


In [42]:
def clean_entry_exit_values(df, max_val, min_val=0):
    """
    takes in a dataframe with the entry/exit diff columns and a max and min val for the entry diff
    returns a dataframe with the crazy values removed
    """
    
    # create mask to remove negative entries and exits or astronomically high differences
    pre_cleaning_rows = df.shape[0]
    cleaning_mask = (df["ENTRIES_DIFF"]>=min_val) & \
                    (df["EXIT_DIFF"]>=min_val) & \
                    (df["ENTRIES_DIFF"]<max_val) & \
                    (df["EXIT_DIFF"]<max_val)
    
    df = df[cleaning_mask]
    post_cleaning_rows = df.shape[0]
    print("You removed {} rows in the cleaning".format(pre_cleaning_rows-post_cleaning_rows))
    return df

In [43]:
combined_df=clean_entry_exit_values(combined_df, 100000)
combined_df.head()

You removed 9654 rows in the cleaning


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY_INT,DAY_STR,ENTRIES_DIFF,EXIT_DIFF
651330,H007,R248,00-00-00,1 AV,L,BMT,09/28/2019,00:00:00,REGULAR,15011834,16775459,2019-09-28 00:00:00,5,Saturday,0.0,0.0
651331,H007,R248,00-00-00,1 AV,L,BMT,09/28/2019,04:00:00,REGULAR,15011834,16775459,2019-09-28 04:00:00,5,Saturday,0.0,0.0
651332,H007,R248,00-00-00,1 AV,L,BMT,09/28/2019,08:00:00,REGULAR,15011834,16775461,2019-09-28 08:00:00,5,Saturday,0.0,2.0
651333,H007,R248,00-00-00,1 AV,L,BMT,09/28/2019,12:00:00,REGULAR,15011834,16775480,2019-09-28 12:00:00,5,Saturday,0.0,19.0
651334,H007,R248,00-00-00,1 AV,L,BMT,09/28/2019,16:00:00,REGULAR,15011834,16775500,2019-09-28 16:00:00,5,Saturday,0.0,20.0


In [44]:
def daily_totals_combined(df):
    
    """
    takes in a dataframe with the entry/exit diff columns
    returns a dataframe with rows of total entries, exits, and cobined for each station in desc order
    """
    
    # show the total entries and exits, it looks much better
    entries_exit_totals = df.groupby(["STATION"])[["ENTRIES_DIFF", "EXIT_DIFF"]].sum()
    
    # cobine the entries and exits and sort to get the most popuklar stations
    entries_exit_totals["COMBINED"] = entries_exit_totals["ENTRIES_DIFF"] + entries_exit_totals["EXIT_DIFF"]
    entries_exit_totals = entries_exit_totals.sort_values(by=["COMBINED"], ascending=False)
    
    return entries_exit_totals

In [45]:
daily_totals = daily_totals_combined(combined_df)
daily_totals

,ENTRIES_DIFF,EXIT_DIFF,COMBINED
STATION,,,
34 ST-PENN STA,4105896.0,3800855.0,7906751.0
GRD CNTRL-42 ST,3423804.0,3307108.0,6730912.0
34 ST-HERALD SQ,2899040.0,2785237.0,5684277.0
23 ST,2902818.0,2182637.0,5085455.0
14 ST-UNION SQ,2539897.0,2403720.0,4943617.0
TIMES SQ-42 ST,2477572.0,2409297.0,4886869.0
FULTON ST,2352961.0,2088237.0,4441198.0
42 ST-PORT AUTH,2281313.0,2036327.0,4317640.0
86 ST,2112051.0,2025804.0,4137855.0
